From Tensor SkFlow: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/skflow/boston.py

## Import

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import datasets, cross_validation, metrics
from sklearn import preprocessing

import chainer.functions as F
import chainer.links as L
from chainer import optimizers, Chain
from commonml.sklearn import ChainerEstimator, MeanSquaredErrorRegressor

import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20

## Load dataset

In [ ]:
boston = datasets.load_boston()
X, y = boston.data, boston.target

## Split dataset into train / test

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y,
    test_size=0.2, random_state=42)

## Scale data (training set) to 0 mean and unit standard deviation.

In [ ]:
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)

## Build 2 layer fully connected DNN with 10, 10 units respectively.

In [ ]:
class Model(Chain):

    def __init__(self, in_size):
        super(Model, self).__init__(l1=L.Linear(in_size, 10),
                                    l2=L.Linear(10, 10),
                                    l3=L.Linear(10, 1),
                                   )

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        h3 = self.l3(h2)
        return h3

regressor = ChainerEstimator(model=MeanSquaredErrorRegressor(Model(X_train.shape[1])),
                             optimizer=optimizers.AdaGrad(lr=0.1),
                             batch_size=100,
                             gpu=0,
                             n_epoch=1000)

## Fit

In [ ]:
regressor.fit(X_train, y_train)

## Predict and score

In [ ]:
score = metrics.mean_squared_error(regressor.predict(scaler.transform(X_test)), y_test)

print('MSE: {0:f}'.format(score))